## Imports et initialisation

In [1]:
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import re
from os import listdir
from os.path import isfile, join
from numpy import asarray
import pandas as pd
import random
import os

In [2]:
dir_path = "D:/Users/leoca/Documents/Cours/Projet_3A/Dataset_couleur"
os.chdir(dir_path)

## One VS Rest : Front

In [3]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.losses import CategoricalCrossentropy
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD

In [4]:
with open('array_entree_224.npy', 'rb') as f:
    X = np.load(f)

with open('array_sortie_224.npy', 'rb') as f:
    y = np.load(f)

In [5]:
train_proportion = int(0.8*len(X))

X_train = X[:train_proportion]
X_test = X[train_proportion:]
y_train = y[:train_proportion]
y_test = y[train_proportion:]

In [8]:
np.shape(X_train)

(4744, 224, 224, 3)

In [9]:
### VGG16
# Initialize the model
model = Sequential()

# layer 1-2: 2 convolutional layers + 1 max-pooling layer 
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu', input_shape = (224,224,3)))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2), strides = 2))

# number of filters and convolutions in each layer:
filters_convs = [(128, 2), (256, 3), (512, 3), (512,3)]

for n_filters, n_convs in filters_convs:
  for _ in np.arange(n_convs):
    model.add(Conv2D(filters = n_filters, kernel_size = (3,3), padding = 'same', activation = 'relu'))
  # max-pooling layer 
  model.add(MaxPooling2D(pool_size = (2,2), strides = 2))
  
model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Dense(16))
model.add(Dense(1, activation='sigmoid'))

# compile the model with a loss function, a metric and an optimization method
opt = SGD(lr = 0.01) # stochastic gradient descent method with learning rate lr = 0.01
model.compile(loss = CategoricalCrossentropy, 
              optimizer = opt, 
              metrics = ['accuracy'])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_14 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                                 
 conv2d_15 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_16 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                  

C:\Users\leoca\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
### Nin
model = Sequential()

# block 1: 
model.add(Conv2D(filters = 96, kernel_size = (11,11), strides = 4, activation = 'relu', input_shape = (224, 224,3)))
model.add(Conv2D(filters = 96, kernel_size = (1,1), activation = 'relu'))
model.add(Conv2D(filters = 96, kernel_size = (1,1),  activation = 'relu'))
model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

# blocks 2-3: 
# paras includes a sequence of parmaters (filters, kernel_size) 
paras = [(256, (5,5)), (384, (3,3))]
for n_filters, kernel_size in paras: 
    model.add(Conv2D(filters = n_filters, kernel_size = kernel_size, padding = 'valid', activation = 'relu'))
    model.add(Conv2D(filters = n_filters, kernel_size = (1,1), activation = 'relu'))
    model.add(Conv2D(filters = n_filters, kernel_size = (1,1), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

model.add(Dropout(0.5))
# block 4:
model.add(Conv2D(filters = 10, kernel_size = (3,3), padding = 'valid', activation = 'relu'))
model.add(Conv2D(filters = 10, kernel_size = (1,1), activation = 'relu'))
model.add(Conv2D(filters = 10, kernel_size = (1,1), activation = 'relu'))

# Global Average pooling layer 
model.add(GlobalAveragePooling2D())
model.add(Dense(1000, activation = 'softmax'))

# compile the model
opt = SGD(lr = 0.01)
model.compile(loss = categorical_crossentropy,
            optimizer = opt, 
            metrics = ['accuracy'])

In [7]:
checkpoint = ModelCheckpoint(dir_path + '/models_VGG16/callback_checkpoint/model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

history = model.fit(X_train, y_train[:, 0], batch_size=25, epochs=5, validation_split=0.1, callbacks=[checkpoint], verbose=1)

Epoch 1/5


TypeError: in user code:

    File "C:\Users\leoca\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\leoca\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\leoca\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\leoca\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\leoca\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "C:\Users\leoca\AppData\Roaming\Python\Python39\site-packages\keras\engine\co